In [1]:
import gradio as gr

from QaS import Embedding, Chunk, CustomLLM, Conversation

e:\Programs\Py\IhmSqr\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


- Extraction du text
- Morcelage du texte
- Récupération du modèle de vectorisation

In [2]:
text = Chunk.getText('./datas.pdf')
chunks = Chunk.createChunks(text, ch_size=1500, ch_over=130)
emb = Embedding.getEmbedding("./LocalModel")

- Création de la base de connaissance

In [3]:
knowledge_base = Conversation.getDatabase(chunk=chunks, embedding=emb)

- Fonction pour obtenir la réponse

In [4]:
def qa(question:str):
    llm = CustomLLM()
    chain = Conversation.loadQA(llm=llm)
    return Conversation.getReponse(chain=chain, data=knowledge_base, query=question)['output_text']

- Création de l'interface avec gradio

In [5]:
query = gr.Interface(
            fn=qa,
            inputs=gr.Textbox(placeholder='Votre question', label='Question'),
            outputs=gr.Textbox(label='Réponse'),
            title='IHM SQR',
            theme=gr.themes.Glass())

Utilisation

In [9]:
if __name__ == '__main__':
    query.close()
    query.launch(debug=True)

Closing server running on port: 7861
